In [ ]:
%matplotlib

import re
import copy
import sys
from enum import Enum
import matplotlib.pyplot as plt
import time
import numpy as np
import random
import plotly.figure_factory as ff
import copy

In [ ]:

def SPT(possible_tasks):
    smallest_duration = sys.maxsize
    best_task = None
    for t in possible_tasks:
        if t.duration < smallest_duration:
            smallest_duration = t.duration
            best_task = t
    return best_task

def LPT(possible_tasks):
    smallest_duration = 0
    best_task = None
    for t in possible_tasks:
        if t.duration > smallest_duration:
            smallest_duration = t.duration
            best_task = t
    return best_task

class STATE(Enum):
    NOT_DONE = '--'
    ON_GOING = '+/-' 
    DONE = '++'
    
class Task:  
    def __init__(self,ressource,duration,job_num):
        self.res = ressource
        self.job_num = job_num
        self.duration = duration
        self.state = STATE.NOT_DONE
        self.start_time = None
        self.possible_start_time = 0 #Utile pour la contruction du Gantt
        
    def reset(self):
        self.state = STATE.NOT_DONE
        self.start_time = None
        self.possible_start_time = 0
        
    def to_string(self):
        return str(self.res) + " " + str(self.duration)

    def start(self,time):
        self.start_time = time
        self.state = STATE.ON_GOING
        
    def start_possible_time(self):
        self.start_time = self.possible_start_time
                
    def update_possible_start_time(self,time):
        if(self.possible_start_time < time):
            self.possible_start_time = time
    
    def update(self,time):
        if(self.state == STATE.ON_GOING):
            if time >= self.start_time + self.duration:
                self.state = STATE.DONE
                return self.res
        return -1
                
    def is_done(self):
        return self.state == STATE.DONE
        
class Job:
    def __init__(self,job_num):
        self.tasks = []
        self.num = job_num
        
    def reset(self):
        for t in self.tasks:
            t.reset()
        
    def init_possible_start(self):
        delay = 0
        for t in self.tasks:
            t.possible_start= delay
            delay = delay + t.duration
        
    def add_task(self,task):
        self.tasks.append(task)
        
    def to_string(self):
        res = "|"
        for t in self.tasks:
            res = res + " " + '{:^6}'.format(t.to_string()) + " |" 
        return res
    
    def get_next_task(self):
        on_going = [task for task in self.tasks if task.state == STATE.ON_GOING]
        if  len(on_going) == 0:
            not_done_tasks = [task for task in self.tasks if task.state == STATE.NOT_DONE]
            if(len(not_done_tasks) != 0):
                return not_done_tasks[0]
            else:
                return None
        
    def start_next_task(self,time):
        t = next(task for task in self.tasks if task.state == STATE.NOT_DONE)
        if t != None:
            t.start(time)
            return t.res
        
    def start_task_at_possible_time(self,task_ind):
        t = self.tasks[task_ind]
        t.start_possible_time()
        delay=t.duration + t.start_time
        for i in range(task_ind+1, len(self.tasks)):
            self.tasks[i].update_possible_start_time(delay)
            delay = delay + self.tasks[i].duration
        return t.res, t.start_time, t.duration+t.start_time
    
    def update_possible_time_by_ressource(self,ressource,time):
        for t in self.tasks:
            if(t.res == ressource):
                t.update_possible_start_time(time)
        
    def update(self,time):
        res_to_free = []
        for t in self.tasks:
            res_to_free.append(t.update(time))
        return res_to_free
            
    def is_done(self):
        done = True
        for t in self.tasks:
            done = done & t.is_done()
        return done
    
    def get_end_time(self):
        return self.tasks[-1].start_time + self.tasks[-1].duration
        
class Instance_heuristic:
    def __init__(self,n_machines,jobs):
        self.n_machines = n_machines
        self.jobs=copy.deepcopy(jobs)
        self.machines_availability = [True]*n_machines
        self.time = 0
        self.order = []

    def get_max_tasks(self):
        nb_max=0
        for j in self.jobs:
            if(len(j.tasks)>nb_max):
                nb_max = len(j.tasks)
        return nb_max
    
    def check_possible_tasks(self):
        possible_tasks = []
        for j in self.jobs:
            next_task = j.get_next_task()
            if(next_task != None):
                if(self.machines_availability[next_task.res]):
                    possible_tasks = possible_tasks + [next_task]
        return possible_tasks
    
    def display_jobs(self):
        max_tasks_per_job = 0
        for j in self.jobs:
            if(max_tasks_per_job < len(j.tasks)):
                max_tasks_per_job = len(j.tasks)
        header=" "
        for t in range(max_tasks_per_job):
            header = header + " " + '{:^6}'.format("R D") + "  " 
        print(header)
        for j in self.jobs:
            print(j.to_string())
            
    def get_state_matrix(self):
        m = np.zeros((len(self.jobs),self.get_max_tasks()))
        for j_num,j in enumerate(self.jobs):
            red_tasks=[]
            orange_tasks=[]
            green_tasks=[]
            for i,t in enumerate(j.tasks):
                if(t.state == STATE.ON_GOING):     
                    m[j_num][i] = 0             
                elif(t.state == STATE.DONE):
                    m[j_num][i] = 1           
                else:
                    m[j_num][i] = -1 
        return m
            
        
    def get_all_tasks(self):
        all_tasks = []
        for j in self.jobs:
            all_tasks = all_tasks + j.tasks
        return all_tasks
    
    def time_forward(self):
        self.time = self.time + 1
        for j in self.jobs:
            res_to_free = j.update(self.time)
            
            for r in res_to_free:
                if(r != -1):
                    self.machines_availability[r] = True
        
            
    def start_job(self,job_num):
        res_used = self.jobs[job_num].start_next_task(self.time)
        self.machines_availability[res_used] = False
        
    def is_done(self):
        done = True
        for j in self.jobs:
            done = done & j.is_done()
        return done
    
    def start_heuristic(self,fct=SPT,rand=0,display=False):
        if(display):
            matrix_state = self.get_state_matrix()
            fig, ax = plt.subplots()
            grid_x=[i+0.5 for i in range(matrix_state.shape[1])]
            ax.set_xticks(grid_x,minor=True)
            grid_y=[i+0.5 for i in range(matrix_state.shape[0])]
            ax.set_yticks(grid_y,minor=True)
            ax.grid(which='minor')
            ax.matshow(matrix_state, cmap='RdYlGn', vmin=-1, vmax=1)
            for i,jb in enumerate(self.jobs):
                for j,t in enumerate(jb.tasks):
                    text = ax.text(j, i, t.res,ha="center", va="center", color='black', fontsize=16)
            plt.xlabel('Tasks')
            plt.ylabel('Jobs')
            
        while not self.is_done():
            possible_tasks = self.check_possible_tasks()
            if(len(possible_tasks) > 0):
                if(random.random()<rand):
                    highest_prio_task = random.choice(possible_tasks)
                else:
                    highest_prio_task = fct(possible_tasks)
                self.start_job(highest_prio_task.job_num)
                self.order.append(highest_prio_task.job_num)
            else:
                self.time_forward()
                
                if(display):
                    matrix_state = self.get_state_matrix()
                    ax.matshow(matrix_state, cmap='RdYlGn', vmin=-1, vmax=1)
                    ax.set_title("TIME : "+str(self.time))
                    plt.pause(0.01)
  


In [ ]:
instances_path = 'JSPLIB/instances/'
instance_path = instances_path + 'ft06'

list_jobs = []
n_machines = 0
n_jobs = 0
# Lecture de l'instance
with open(instance_path,'r') as f:
    num_line = 0
    for line in f:
        if(line[0] != '#'):
            
            re_line = re.sub(' +',';',line)
            re_line = re.sub('\\n','',re_line)
            split_line = re_line.split(';')
            if (num_line == 0):
                n_machines = int(split_line[1])
                n_jobs = int(split_line[0])
            else:
                list_jobs.append(split_line)
            num_line = num_line+1
for j in list_jobs:
    print(j)

In [ ]:
# Création des jobs à partir des listes précédentes
jobs = []
for i in range(n_machines):
    job = Job(i)
    for j in range(0,2*n_jobs,2):
        job.add_task(Task(int(list_jobs[i][j]),int(list_jobs[i][j+1]),i))
    jobs.append(job)
    

In [ ]:

instance = Instance_heuristic(n_machines,jobs)
print("Nombre de machines : ",instance.n_machines)
print('Nombre de jobs : ',len(instance.jobs))
print('Taches : ')
instance.display_jobs()

In [ ]:
instance.start_heuristic(fct=LPT,rand=1,display=True)
print(instance.order)

In [ ]:
print(instance.time)

In [ ]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

class Repetition_order():
    def __init__(self,order,jobs):
        self.order = order
        self.voisinage = []
        self.duration = None
        self.jobs = copy.deepcopy(jobs)
        self.compute()

    def compute(self):
        job_task_index = [0] * len(self.jobs)
        for j in self.jobs:
            j.reset()
            j.init_possible_start()
        for i in self.order:
            ressource_used,start_time, end_time = self.jobs[i].start_task_at_possible_time(job_task_index[i])
            job_task_index[i] = job_task_index[i] + 1
            for j in self.jobs:
                j.update_possible_time_by_ressource(ressource_used,end_time)
        self.duration = self.get_end_time()
   
    def display_gantt(self):
        colors = {}
        list_colors = ['dodgerblue','seagreen','sienna','turquoise','orchid','gold','black','white','red'
                       ,'blue','green','yellow','purple']
        fig = go.Figure(
            layout = {
                'barmode': 'stack',
                'xaxis': {'automargin': True},
                'yaxis': {'automargin': True}}#, 'categoryorder': 'category ascending'}}
        )
        for j in self.jobs:
            for t in j.tasks:
                show_legend = False
                if(str(t.res) not in colors.keys()):
                    color_picked = list_colors[0]
                    list_colors.remove(color_picked)
                    colors[str(t.res)] =  color_picked
                    show_legend = True

                fig.add_bar(x=[t.duration],
                    y=['Job ' + str(j.num)],
                    base=t.start_time,
                    orientation='h',
                    showlegend=show_legend,
                    name=str(t.res),
                    marker={'color': colors[str(t.res)]}) 
        iplot(fig)

        
    def get_end_time(self):
        max_jobs = []
        for j in self.jobs:
            max_jobs.append(j.get_end_time())
        return max(max_jobs)
            
    def get_swapped(self,ind1,ind2):
        res = self.order.copy()
        c = res[ind1]
        res[ind1] = res[ind2]
        res[ind2] = c
        return res
        
    def reset_voisinage(self):
        self.voisinage = []
        
    def set_2swap_voisinage(self):
        for i in range(len(self.order)):
            for j in range(i,len(self.order)):
                self.voisinage.append(Repetition_order(self.get_swapped(i,j),self.jobs))

    def set_block_swap_voisinage(self,nb_cuts,nb_voisins):
        for no_voisin in range(nb_voisins):
            random_cut = random.sample(range(2, len(self.order)), nb_cuts)
            random_cut.append(0)
            random_cut.append(len(self.order))
            random_cut.sort()
            blocks = []
            for i in range(len(random_cut)-1):
                blocks.append(self.order[random_cut[i]:random_cut[i+1]])

            random.shuffle(blocks)
            new_voisin = []
            for b in blocks:
                new_voisin = new_voisin+b

            self.voisinage.append(Repetition_order(new_voisin,self.jobs))
            
        

    def get_best_voisin(self):
        if(len(self.voisinage)>=1):
            best_voisin = self
            best_perf = self.duration
            for v in self.voisinage:
                if(v.duration <= best_perf):
                    best_voisin = v
                    best_perf = v.duration
            return best_voisin
        else: 
            return None


In [ ]:
        
rep_order = Repetition_order(instance.order,instance.jobs) 
print('Repetition order from heuristic : ', rep_order.order)
rep_order.display_gantt()
print('Time : ',rep_order.duration)
print('=============  VOISINAGE  ===============')
best_voisin = rep_order
nb_iter = 10
i=0
previous_order = None
while i<nb_iter and previous_order!=best_voisin.order:
    previous_order = best_voisin.order
    best_voisin.reset_voisinage()
    best_voisin.set_2swap_voisinage()
    best_voisin = best_voisin.get_best_voisin()
    print("==========")
    print("Itération ",i)
    print("Meilleur voisin : ", best_voisin.order)
    print("Durée : ", best_voisin.duration)
    i=i+1

best_voisin.display_gantt()

### Essaie pour trouver l'optimum 

In [ ]:
instance_pour_opt = Instance_heuristic(n_machines,jobs)
instance_pour_opt.start_heuristic(fct=LPT,rand=0.5,display=False)
order_to_start_with = [2, 1, 2, 4, 3, 2, 5, 3, 1, 4, 5, 2, 1, 5, 0, 3, 3, 0, 2, 4, 1, 0, 5, 3, 0, 4, 1, 5, 3, 5, 2, 1, 4, 0, 4, 0]

rep_order = Repetition_order(order_to_start_with,instance_pour_opt.jobs)
print('Repetition order from heuristic : ', rep_order.order)
print('Time : ',rep_order.duration)
rep_order.display_gantt()


In [ ]:
print('=============  VOISINAGE  ===============')
best_voisin = rep_order
nb_iter = 10
i=0
previous_rep = None
while i<nb_iter and previous_rep!=best_voisin.order:
    previous_rep = best_voisin.order
    best_voisin.reset_voisinage()
    best_voisin.set_2swap_voisinage()
    best_voisin = best_voisin.get_best_voisin()
    print("==========")
    print("Itération ",i)
    print("Meilleur voisin : ", best_voisin.order)
    print("Durée : ", best_voisin.duration)
    i=i+1

best_voisin.display_gantt()

 ### Nouveaux voisins

In [ ]:
instance_pour_opt = Instance_heuristic(n_machines,jobs)
instance_pour_opt.start_heuristic(fct=LPT,rand=0.5,display=False)

rep_order = Repetition_order(instance_pour_opt.order,instance_pour_opt.jobs)
print('Repetition order from heuristic : ', rep_order.order)
print('Time : ',rep_order.duration)
rep_order.display_gantt()
print('=============  VOISINAGE  ===============')
best_voisin = rep_order
nb_iter = 10
i=0
previous_rep = None
while i<nb_iter and previous_rep!=best_voisin.order:
    previous_rep = best_voisin.order
    best_voisin.reset_voisinage()
    best_voisin.set_block_swap_voisinage(3,10000)
    best_voisin.set_2swap_voisinage()
    best_voisin = best_voisin.get_best_voisin()
    print("==========")
    print("Itération ",i)
    print("Meilleur voisin : ", best_voisin.order)
    print("Durée : ", best_voisin.duration)
    i=i+1

best_voisin.display_gantt()